In [11]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [12]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [13]:
import os
from pyngrok import ngrok

In [14]:
ngrok.kill()

In [15]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://jeanice-unmonetary-suzanne.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://jeanice-unmonetary-suzanne.ngrok-free.dev


True

In [16]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        response_modalities=["TEXT"],
    )
)

In [17]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [18]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Ming Hsin University of Science and Technology, MHUST）是一所位於台灣新竹縣新豐鄉的**私立科技大學**。

以下是其主要簡介：

1.  **創立背景與沿革：**
    *   創立於**1966年**，前身為「明新工業專科學校」。
    *   歷經多年發展，於1997年改制為「明新技術學院」，並於**2002年正式升格為「明新科技大學」**。

2.  **教育理念與特色：**
    *   學校以「**誠信、創新、卓越**」為核心價值。
    *   致力於培育具備**實作能力、創新思維與產業即戰力**的專業技術人才。
    *   課程設計強調**理論與實務結合**，重視學生動手做的能力，並輔導考取相關專業證照。
    *   **產學合作**緊密，積極與各行各業建立合作關係，為學生提供豐富的實習與就業機會。

3.  **地理位置優勢：**
    *   地理位置優越，**緊鄰新竹科學園區**，這使得學校在電子、半導體、資訊科技等相關產業的產學合作上具有得天獨厚的優勢，畢業生在當地就業市場上具有競爭力。

4.  **學院與學制：**
    *   設有**工程學院、管理學院、服務事業學院、設計學院**等四大院。
    *   提供學士班（四技、二技）、碩士班等多層次學制，涵蓋電機、電子、資工、機械、土木、工管、企管、行銷、財金、休閒事業、旅館管理、運動管理、多媒體設計、時尚造型等多元領域。

5.  **發展願景：**
    *   明新科大以成為**技職教育典範**為目標，持續精進教學品質與研究發展，培養國家與社會所需的創新專業人才，並積極推動國際化，擴展學生的國際視野。

總結來說，明新科技大學是一所位於台灣高科技產業重鎮的實務導向型科技大學，以培育具備產業競爭力的專業人才為主要特色。


In [19]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是 **劉國偉 教授**。


In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[{"type":"message","message":{"type":"text","id":"594674802318114817","quoteToken":"Az23O7CZTeP9V6kAnxhFhyUxKyNMcM06DwCGmV70aCAFBtia9vzSHY-o8XApfPWt_LtyN3VgDUJDxVcKh7uZffdTBRWfdrDibha2bfuHlDLVURWEZWkCeuR7kKpZOwVt1U0nHLqu0jqzyoicPLcxCA","markAsReadToken":"15anpnLCxK9mG4sZwzXrT0x_6vnutKeA-KvTl62bBScC83_xWdI14w1uSXTS2rP-DJqRXsO3H1BQFawqMSxaMIw2IERbymAf7RqoI9k0YjS0KhE2ZCP3Xbdt1Bod9kCkV3wXq5JRBiVjPeO3aJiZrq7dlpCJPvRbN2EWPtz77zFKnkEi1S02LxoZ5XeeJuGWfX3HgkWcZXv9Ot98R26x6g","text":"AI介紹一下明新科大(五十字以内)"},"webhookEventId":"01KDX6DAWF27DMNS8EHSWRWQA0","deliveryContext":{"isRedelivery":false},"timestamp":1767285107498,"source":{"type":"user","userId":"U708a8161d116dd816aab6ebb0ad0157c"},"replyToken":"35a5074b74d446d6b9d2c1c840154687","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:31:48] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[{"type":"message","message":{"type":"text","id":"594674829195215359","quoteToken":"PiyJZ-Z3CcUBmHNlWDtIZB4Hx9o9-LQwRPIu817lRnZ0cGH-KJv1DVjgew476sxjA97URwQwybOPqWQ1Lf-nBpyxYarNc0aAc24w2VzZAIpDgbW_iU4rZotKZvySX5f958rKEbCfY6FiHK3XcaGg4w","markAsReadToken":"tmiv3Yex2UpgQXJ-oocQ-sajKwAaYPMIxdZCkWB_ulOhYuHZY--b8lKQxD-AnDKishHFxnf_xqioEKvkGsw_TzbVFBEleg2OyDoGfrfFqQaRN78v9XJdbv7VGoVZKqw_Oh15rHSdllYnCtUgjikGyqd8Ny2iklKNbXoMQLW8EGVT2LvHLFzMjvkot9n-ABLINiU_B6NVmGihJlk9FVaFhg","text":"AI 介紹一下明新科大(五十字以内)"},"webhookEventId":"01KDX6DTTTHMGWDJRKG7RBHXYZ","deliveryContext":{"isRedelivery":false},"timestamp":1767285123527,"source":{"type":"user","userId":"U708a8161d116dd816aab6ebb0ad0157c"},"replyToken":"01d9d27b7c5e435ea1f9a5d2789b7eba","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:32:08] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"Ufe0037d5ecf7ada870601eb0bc93c619","events":[{"type":"message","message":{"type":"text","id":"594674900883734841","quoteToken":"ynxKyeIjLx0sDWyGn2Uh8rDE0OseHlTy6U6v6DAXWWlwyTMTebRpMnjYc_F4cQL0rHmIMnFnQ1BfBr9zqv4wV9V5hDWgTj9jnkPDtd-YtBNEHw6u0f_oAcshhxZHihe7AxJPaVK_arbkqvty963bhw","markAsReadToken":"zTU3ZA0NRSsv4nt06qNgyGpnmdj8YQSkubrcpVaD8TcavlM-xz1D1u-6pMTqs-UuQHfnGh5Ti4trwefhSBQccFVSQSVABUPl-QADX3_RkcAo-jKlVxN-ZUrveeqfds9LgAAP0AqOZEQ02CyjTTLUQWREb4FutOSoG4KZUW6VLFWNaHKIamGMn9mBQ5JO74ZgWOKhpMZ7ozIy-2Z7hYLVqg","text":"AI 校長是誰?"},"webhookEventId":"01KDX6F4NTRABP1ZBMQ9MYWR39","deliveryContext":{"isRedelivery":false},"timestamp":1767285166417,"source":{"type":"user","userId":"U708a8161d116dd816aab6ebb0ad0157c"},"replyToken":"5cdef33df7854cb79fd6bf252d26f3c5","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [01/Jan/2026 16:32:48] "POST / HTTP/1.1" 200 -
